In [1]:
import pandas as pd
import datetime
import requests, io
import numpy as np

In [2]:
#importação da base
url = 'http://www.anatel.gov.br/dadosabertos/PDA/PBLE/PBLE.csv'
pble = pd.read_csv(url, sep=';', encoding='latin-1')

In [3]:
#renomeando variáveis
pble.rename(columns={'Prestadora':'empresa', 'Nº INEP': 'id_escola', 'UF':'sigla_uf', 'Municipio':'municipio', 'Situação':'situacao',
              'Tipo de Obrigação':'programa', 'Velocidade de Acesso Instalada':'conexao', 'Tecnologia':'tecnologia',
             'Nome da Escola':'nome_escola', 'Tipo Escola':'rede', 'Data de Ativação':'data_ativacao'}, inplace=True)

In [4]:
#exclusão de variaveis unicas: todas instaladas, programa PBLE, mes/ano de atualizacao dos dados
pble.drop(['situacao', 'programa', 'Mes', 'Ano', 'nome_escola'], axis=1, inplace=True)

In [5]:
#transformar pra formato data
pble['data_ativacao']=pd.to_datetime(pble['data_ativacao'].astype(str), format='%d/%m/%Y')

In [6]:
#criacao de colunas separadas por ano/mes/dia
pble['ano'] = pble['data_ativacao'].dt.year
pble['mes'] = pble['data_ativacao'].dt.month
pble['dia'] = pble['data_ativacao'].dt.day

In [7]:
#retirar [*] de algumas observações
pble['id_escola'] = pble['id_escola'].apply(lambda x: str(x).replace('[*]',''))

In [8]:
sigla_uf_dic ={'RO':11, 'AC':12, 'AM':13, 'RR': 14, 'PA':15, 'AP':16, 'TO':17,
               'MA':21, 'PI':22, 'CE':23, 'RN':24, 'PB':25, 'PE':26, 'AL': 27, 'SE':28, 'BA':29,
               'MG':31, 'ES':32, 'RJ':33, 'SP':35,
               'PR': 41, 'SC':42, 'RS':43,
               'MS':50, 'MT':51, 'GO':52, 'DF':53}
pble['id_uf'] = pble['sigla_uf'].map(sigla_uf_dic)

In [14]:
pble.replace(regex={r'São Luís do Paraitinga': 'São Luiz do Paraitinga', "Pau D'Arco": "Pau d'Arco", 'Augusto Severo':'Augusto Severo (Campo Grande)',
                    'Januário Cicco':'Januário Cicco (Boa Saúde)', 'Quixaba':'Quixabá', 'São Caitano':'São Caetano', 'Graccho Cardoso':'Gracho Cardoso',
                    "Pau d'Arco do Piauí":"Pau D'Arco do Piauí", "Olhos-d'Água":"Olhos d'Água"}, inplace=True)

In [10]:
#inclusão do id_municipio (código do ibge) pelo nome do municipio
url = 'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv'
read_data = requests.get(url).content
read_data

b"\xef\xbb\xbfcodigo_ibge,nome,latitude,longitude,capital,codigo_uf\n5200050,Abadia de Goi\xc3\xa1s,-16.7573,-49.4412,0,52\n3100104,Abadia dos Dourados,-18.4831,-47.3916,0,31\n5200100,Abadi\xc3\xa2nia,-16.197,-48.7057,0,52\n3100203,Abaet\xc3\xa9,-19.1551,-45.4444,0,31\n1500107,Abaetetuba,-1.72183,-48.8788,0,15\n2300101,Abaiara,-7.34588,-39.0416,0,23\n2900108,Aba\xc3\xadra,-13.2488,-41.6619,0,29\n2900207,Abar\xc3\xa9,-8.72073,-39.1162,0,29\n4100103,Abati\xc3\xa1,-23.3049,-50.3133,0,41\n4200051,Abdon Batista,-27.6126,-51.0233,0,42\n1500131,Abel Figueiredo,-4.95333,-48.3933,0,15\n4200101,Abelardo Luz,-26.5716,-52.3229,0,42\n3100302,Abre Campo,-20.2996,-42.4743,0,31\n2600054,Abreu e Lima,-7.90072,-34.8984,0,26\n1700251,Abreul\xc3\xa2ndia,-9.62101,-49.1518,0,17\n3100401,Acaiaca,-20.359,-43.1439,0,31\n2100055,A\xc3\xa7ail\xc3\xa2ndia,-4.94714,-47.5004,0,21\n2900306,Acajutiba,-11.6575,-38.0197,0,29\n1500206,Acar\xc3\xa1,-1.95383,-48.1985,0,15\n2300150,Acarape,-4.22083,-38.7055,0,23\n2300200,A

In [15]:
id_municipio = pd.read_csv(io.StringIO(read_data.decode('utf-8')))
id_municipio.drop(['capital', 'latitude', 'longitude'], axis=1, inplace=True)

In [16]:
pble = pd.merge(pble, id_municipio,
                how='left', left_on=['municipio', 'id_uf'],
                right_on=['nome', 'codigo_uf'])

In [23]:
pble['codigo_ibge'].loc[(pble['sigla_uf'] =='TO') & (pble['municipio']=="Pau d'Arco") & (pble['codigo_ibge'].isnull())]=1716307
pble['codigo_ibge'].loc[(pble['sigla_uf'] =='PI') & (pble['municipio']=="Pau d'Arco do Piauí") & (pble['codigo_ibge'].isnull())]=2207793
pble['codigo_ibge'].loc[(pble['sigla_uf'] =='PE') & (pble['municipio']=="Quixabá") & (pble['codigo_ibge'].isnull())]=2611533

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [28]:
#organização das variaveis
pble.drop(['municipio', 'data_ativacao','id_uf', 'nome', 'codigo_uf'], axis=1, inplace=True)
pble.rename(columns={'codigo_ibge':'id_municipio'}, inplace=True)
pble = pble[['ano', 'mes', 'dia', 'sigla_uf', 'id_municipio', 'rede', 'id_escola', 'empresa', 'tecnologia', 'conexao']]

In [39]:
#salvar arquivos no Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
#exportação para csv
pble.to_csv('/content/drive/MyDrive/br_anatel/banda_larga_fixa/output/banda_larga_pble', index=False)